In [1]:
from pyspark.sql import SparkSession

"""make an instance of a SparkSession called 'spark'."""
spark = SparkSession.builder.master('local').getOrCreate()


22/07/01 12:41:47 WARN Utils: Your hostname, fossa-dsa-001 resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp3s0)
22/07/01 12:41:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/01 12:41:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [33]:
# set our path
data_file = 'data/coffee.csv'
# set schema
customSchema = 'date date, open float, high float, low float, close float, volume float, currency string'
# read our csv into a spark df
sdf = spark.read.csv(data_file, schema = customSchema, header = True)
sdf.describe

<bound method DataFrame.describe of DataFrame[date: date, open: float, high: float, low: float, close: float, volume: float, currency: string]>

In [70]:
# Add a column to the DataFrame where the values are the difference between 'Open' and 'Close'.
from pyspark.sql.functions import *
sdf = sdf.withColumn('open2close',
                (sdf.open - sdf.close))
# Add a column to the DataFrame where the values are the difference between 'High' and 'Low'.
sdf = sdf.withColumn('high2low',
                (sdf.high - sdf.close))
# Add a column to the DataFrame where the values are 'True' if the volume for that day was 100 or above, and otherwise 'False'.
sdf = sdf.withColumn('did_well',
                     when((sdf.volume > 99), lit(True))
                     .otherwise(lit(False)))
# add another column that contains the absolute values of the numbers in that column.
sdf = sdf.withColumn('day_abs', abs(sdf.open2close))
sdf.printSchema()

root
 |-- date: date (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- currency: string (nullable = true)
 |-- open2close: float (nullable = true)
 |-- high2low: float (nullable = true)
 |-- did_well: boolean (nullable = false)
 |-- day_abs: float (nullable = true)



In [52]:
net_sales = mean(sdf.open + sdf.high + sdf.low + sdf.close) * sdf.volume
# print(net_sales)


In [80]:
# Find the average of the values in the column that has the absolute values of the difference between 'Open' and 'Close'.
sdf.select(avg(sdf.day_abs)).show()
# Get the count of values where the 'Volume' was less than 100.
print(sdf.filter(sdf.volume == False).count())
# Find the average 'Open' value.
sdf.select(avg(sdf.open)).show()
# Get the highest 'High' value.
sdf.select(max(sdf.high)).show()


+------------------+
|      avg(day_abs)|
+------------------+
|1.7606027822995378|
+------------------+

201
+-----------------+
|        avg(open)|
+-----------------+
|126.0496775257701|
+-----------------+

+---------+
|max(high)|
+---------+
|   306.25|
+---------+



In [81]:
sdf.write.parquet('data/coffequet.parquet')